In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import requests
import sqlite3
import datetime
df = pd.DataFrame({"MLS": [],"Street": [], "City":[],"List Price":[],"Bedrooms":[],"Bathrooms":[],"Square Footage":[],"Date":[]})


In [2]:
def scraping(dataframe):
    page=requests.get("http://www.slocountyhomes.com/newlistex.php")
    data=BeautifulSoup(page.text,"html.parser")
    i=0
    date=""
    date="NA"
    MLS=""
    Street=""
    City=""
    List=""
    Bed=""
    Bath=""
    Sq=0
    Date=""
    
    for table in data.find_all('td'):
        if len(table.text.split("November")) != 1:
            date = table.text
        elif table.text.strip() == "":
            if (i != 0):
                # Found missing data
                i += 1
                if (i == 7):
                    i = 0
                    #dataframe.loc[len(dataframe)]=[Bath,Bed,City,date,List,MLS,Sq,Street]
        else:
            if i == 0:
                MLS = table.text.strip()
                i += 1
            elif i == 1:
                Street = sqlize_string(table.text.strip())
                i += 1
            elif i == 2:
                City = sqlize_string(table.text.strip())
                i += 1
            elif i == 3:
                List = int(table.text.strip()[1:].replace(",",""))
                i += 1
            elif i == 4:
                Bed = int(table.text.strip())
                i += 1
            elif i == 5:
                Bath = int(table.text.strip())
                i += 1
            elif i == 6:
                Sq = int(table.text.strip())
                i = 0
                dataframe.loc[len(dataframe)]=[Bath,Bed,City,date,List,MLS,Sq,Street]
                
    dataframe=dataframe.drop_duplicates()
    return dataframe

In [3]:
def sqlize_string(string):
    return "'" + string + "'"

Lot size most important in prediction 


In [4]:
df=scraping(df)

In [11]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,1073005,4,3,Arroyo Grande,1351 Newport,849000.0,3076.0,276.007802,0,None,None,1988
1,2,1072924,3,2,Atascadero,6200 Buena,399000.0,1400.0,285.000000,0,None,None,1988
2,3,1073010,3,3,Grover Beach,1239 Aberdeen,529900.0,1894.0,279.778247,0,None,None,1988
3,4,1072942,3,2,Los Osos,2570 Rodman,779000.0,2315.0,336.501080,0,None,None,1988
4,5,1073009,3,1,Morro Bay,350 Rennell,429000.0,955.0,449.214660,0,None,None,1988
5,6,1072998,3,2,Nipomo,1010 Redberry,615000.0,1890.0,325.396825,0,None,None,1988
6,7,1073004,2,2,Paso Robles,4320 Cascade,176000.0,1632.0,107.843137,0,None,None,1988
7,8,1073018,4,3,Paso Robles,5672 Silverado,505000.0,2097.0,240.820219,0,None,None,1988
8,9,1072999,7,7,Paso Robles,2710 Geneseo,699000.0,3198.0,218.574109,0,None,None,1988
9,10,1072980,4,2,San Miguel,1871 L,350000.0,1547.0,226.244344,0,None,None,1988


In [6]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
month_map = {key:int(val) for key,val in zip(months, range(1,13))}
month_map

{'April': 4,
 'August': 8,
 'December': 12,
 'February': 2,
 'January': 1,
 'July': 7,
 'June': 6,
 'March': 3,
 'May': 5,
 'November': 11,
 'October': 10,
 'September': 9}

In [7]:
conn = sqlite3.connect('slo_housing.db')
print("Opened database successfully");
stmt = "INSERT INTO HOUSES (MLS_ID,BED,BATH,CITY, ADDRESS, LIST_PRICE, SQ_FOOTAGE, PRICE_PER_SQFT, LIST_DATE) \
      VALUES ({}, {}, {}, {}, {}, {}, {}, {}, {})"
for idx, row in df.iterrows():

    conn.execute(stmt.format(row['MLS'], row['Bedrooms'], row['Bathrooms'], row['City'],
                             row['Street'], row['List Price'], row['Square Footage'],
                             int(row['List Price']) / int(row['Square Footage']), 
                             datetime.date(2016, month_map[row['Date'].split()[0]], int(row['Date'].split()[1]))
                            ));
    
    
conn.commit()
print("Records created successfully");
conn.close()

Opened database successfully
Records created successfully


In [8]:
conn = sqlite3.connect('slo_housing.db')
check = conn.execute("SELECT * FROM HOUSES")

In [9]:
df = pd.DataFrame(check.fetchall())

def get_query(query):
    var_names = ['MLS Key','Num Bedrooms','Num Bathrooms', 'City', 'Address', 'Price',
                'Sqr Footage', 'Unknown1', 'Unknown2', 'Unknown3', 'Unknown4', 'Year Sold']
    conn = sqlite3.connect('slo_housing.db')
    df = pd.DataFrame(conn.execute(query).fetchall())
    del df[0]
    df.columns = var_names
    return df
    

In [10]:
get_query('SELECT * FROM HOUSES').head()

,MLS Key,Num Bedrooms,Num Bathrooms,City,Address,Price,Sqr Footage,Unknown1,Unknown2,Unknown3,Unknown4,Year Sold
0,1073005,4,3,Arroyo Grande,1351 Newport,849000.0,3076.0,276.007802,0,None,None,1988
1,1072924,3,2,Atascadero,6200 Buena,399000.0,1400.0,285.000000,0,None,None,1988
2,1073010,3,3,Grover Beach,1239 Aberdeen,529900.0,1894.0,279.778247,0,None,None,1988
3,1072942,3,2,Los Osos,2570 Rodman,779000.0,2315.0,336.501080,0,None,None,1988
4,1073009,3,1,Morro Bay,350 Rennell,429000.0,955.0,449.214660,0,None,None,1988
